In [1]:
%load_ext autoreload
%autoreload 2
# Sanity check on implementation of 
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import time
# Set a limit on the memory usage of the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        print(e)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-03-25 12:35:21.712654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-25 12:35:21.712717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-25 12:35:21.712732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 12:35:21.722779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Ensure that log prob calculations are the same as tfp implementation & faster

In [2]:
# Define your array (as a TensorFlow tensor)
n_rows = 10000 # i.e., number of voxels 
n_cols = 200 # i.e., number of samples in timeseries
num_iterations = 1000
data = tf.constant(np.random.rand(n_rows, n_cols), dtype=tf.float32)
scale_values = tf.constant(np.random.rand(n_rows, 1), dtype=tf.float32)
dof_values = tf.constant(np.random.rand(n_rows, 1), dtype=tf.float32)

2025-03-25 12:35:31.682310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0, compute capability: 3.5


#### testing calculate_log_prob_gauss_loc0

In [3]:


from braincoder.utils.math import calculate_log_prob_gauss_loc0
for device in ['GPU', 'CPU']:
    print(f'Trying with {device}')
    with tf.device(f'/{device}:0'):
        # Timing the custom log probability calculation
        start_time = time.time()
        for _ in range(num_iterations):
            calculate_log_prob_gauss_loc0(data, scale_values)  # Now correctly broadcasted
        time_custom = time.time() - start_time
        output_custom = calculate_log_prob_gauss_loc0(data, scale_values)
        # Timing the TFP log probability calculation
        start_time = time.time()
        for _ in range(num_iterations):
            normal_dist = tfp.distributions.Normal(loc=0.0, scale=scale_values)  # Correct shape
            normal_dist.log_prob(data)  # Correct shape
        time_tfd = time.time() - start_time
        output_tfd = normal_dist.log_prob(data)
        # Print the results
        print(f"Custom log probability calculation time over {num_iterations} iterations: {time_custom:.6f} seconds")
        print(f"TFP log probability calculation time over {num_iterations} iterations: {time_tfd:.6f} seconds")
        print(f"Custom method is  {time_tfd/time_custom:.3f} x faster")
        print(f"     tfd,        custom")
        for i1 in range(3):
            for i2 in range(3):
                print(f'{output_tfd[i1,i2]:10.3f}, {output_custom[i1,i2]:10.3f}')


Trying with GPU
Custom log probability calculation time over 1000 iterations: 1.022761 seconds
TFP log probability calculation time over 1000 iterations: 1.914832 seconds
Custom method is  1.872 x faster
     tfd,        custom
    -1.394,     -1.394
    -0.469,     -0.469
    -0.577,     -0.577
    -1.190,     -1.190
    -1.094,     -1.094
   -19.759,    -19.759
     0.118,      0.118
     0.127,      0.127
     0.120,      0.120
Trying with CPU
Custom log probability calculation time over 1000 iterations: 2.545232 seconds
TFP log probability calculation time over 1000 iterations: 3.609536 seconds
Custom method is  1.418 x faster
     tfd,        custom
    -1.394,     -1.394
    -0.469,     -0.469
    -0.577,     -0.577
    -1.190,     -1.190
    -1.094,     -1.094
   -19.759,    -19.759
     0.118,      0.118
     0.127,      0.127
     0.120,      0.120


#### Testing calculate_log_prob_t

In [4]:
from braincoder.utils.math import calculate_log_prob_t
for device in ['GPU', 'CPU']:
    print(f'Trying with {device}')
    with tf.device(f'/{device}:0'):
        # Timing the custom log probability calculation
        start_time = time.time()
        for _ in range(num_iterations):
            calculate_log_prob_t(data, scale_values, dof_values)  # Now correctly broadcasted
        time_custom = time.time() - start_time
        output_custom = calculate_log_prob_t(data, scale_values, dof_values)

        # Timing the TFP log probability calculation
        start_time = time.time()
        for _ in range(num_iterations):
            t_dist = tfp.distributions.StudentT(df=dof_values, loc=0.0, scale=scale_values)  # Correct shape
            t_dist.log_prob(data)  # Correct shape
        time_tfd = time.time() - start_time
        output_tfd = t_dist.log_prob(data)

        # Print the results
        print(f"Custom log probability calculation time over {num_iterations} iterations: {time_custom:.6f} seconds")
        print(f"TFP log probability calculation time over {num_iterations} iterations: {time_tfd:.6f} seconds")
        print(f"Custom method is  {time_tfd/time_custom:.3f} x faster")
        print(f"     tfd,        custom")
        for i1 in range(3):
            for i2 in range(3):
                print(f'{output_tfd[i1,i2]:10.3f}, {output_custom[i1,i2]:10.3f}')


Trying with GPU
Custom log probability calculation time over 1000 iterations: 3.203720 seconds
TFP log probability calculation time over 1000 iterations: 9.060082 seconds
Custom method is  2.828 x faster
     tfd,        custom
    -1.905,     -1.905
    -0.840,     -0.840
    -1.064,     -1.064
    -1.005,     -1.005
    -0.972,     -0.972
    -2.781,     -2.781
    -0.352,     -0.352
    -0.323,     -0.323
    -0.347,     -0.347
Trying with CPU
Custom log probability calculation time over 1000 iterations: 7.788781 seconds
TFP log probability calculation time over 1000 iterations: 13.588880 seconds
Custom method is  1.745 x faster
     tfd,        custom
    -1.905,     -1.905
    -0.840,     -0.840
    -1.064,     -1.064
    -1.005,     -1.005
    -0.972,     -0.972
    -2.781,     -2.781
    -0.352,     -0.352
    -0.323,     -0.323
    -0.347,     -0.347


# Check GP dists

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import time
import numpy as np

from braincoder.bprf_mcmc import GPdists, mds_embedding, compute_euclidean_distance_matrix
tfd = tfp.distributions


# Create a dummy symmetric distance matrix.
n_vx = 50
rand = tf.random.uniform((n_vx, n_vx), minval=0, maxval=1, dtype=tf.float64)
dists = (rand + tf.transpose(rand)) / 2.0
dists = dists - tf.linalg.diag(tf.linalg.diag_part(dists))

# Set hyperparameters.
gp_variance = tf.constant(1.0, dtype=tf.float32)
gp_lengthscale = tf.constant(1.0, dtype=tf.float32)
gp_mean = tf.constant(0.0, dtype=tf.float32)
gp_nugget = tf.constant(0.1, dtype=tf.float32)

# Instantiate GPdists with fixed hyperparameters.
gp_fixed = GPdists(
    dists,
    fixed_params=True,
    full_norm=True, 
    gp_variance=gp_variance,
    gp_lengthscale=gp_lengthscale,
    gp_mean=gp_mean,
    gp_nugget=gp_nugget,
    psd_control='euclidean',
    dists_dtype=tf.float64, # for cholesky... 
    kernel='RBF',
)

# For comparison, get the covariance matrix computed by GPdists.
cov_matrix = gp_fixed.cov_matrix

# Create a random parameter vector (matching the number of vertices).
parameter = tf.random.normal([n_vx], dtype=tf.float32)

# Warm-up (to compile tf.function graphs)
_ = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)

# Compute log probability using the fixed TF method.
log_prob_fixed_tf = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)

# Compute log probability using the fixed precision method.
log_prob_fixed_prec = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)

# For comparison, compute the log probability "from scratch" using the full Gaussian formula.

print("Log probability (Fixed TF):       ", log_prob_fixed_tf.numpy())
print("Log probability (Fixed Precision):  ", log_prob_fixed_prec.numpy())
# print("Log probability (Full formula):     ", log_prob_full.numpy())


Embedding in Euclidean space...
Precomputing covariance matrix...
Precomputing Cholesky decomposition...
Log probability (Fixed TF):        -179.84727
Log probability (Fixed Precision):   -179.84727


In [37]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import time
import numpy as np

from braincoder.bprf_mcmc import GPdists, mds_embedding, compute_euclidean_distance_matrix
tfd = tfp.distributions
full_norm = True
# ------------------------------
# Setup: create a dummy distance matrix.
# ------------------------------
n_vx = 50
rand = tf.random.uniform((n_vx, n_vx), minval=0, maxval=1, dtype=tf.float64)
dists = (rand + tf.transpose(rand)) / 2.0
dists = dists - tf.linalg.diag(tf.linalg.diag_part(dists))

# ------------------------------
# Hyperparameters
# ------------------------------
# Note: GPdists expects the distance tensor to be tf.float64 (for Cholesky) so we set that,
# but the parameter vector itself is in tf.float32.
gp_variance = tf.constant(1.0, dtype=tf.float32)
gp_lengthscale = tf.constant(1.0, dtype=tf.float32)
gp_mean = tf.constant(0.0, dtype=tf.float32)
gp_nugget = tf.constant(0.1, dtype=tf.float32)

# ------------------------------
# Instantiate GPdists: fixed and unfixed versions.
# ------------------------------
# Fixed instance: precomputes covariance matrix, Cholesky, and precision.
gp_fixed = GPdists(
    dists,
    fixed_params=True,
    full_norm=full_norm,  # Option for full normalization in precision-based log_prob.
    gp_variance=gp_variance,
    gp_lengthscale=gp_lengthscale,
    gp_mean=gp_mean,
    gp_nugget=gp_nugget,
    psd_control='euclidean',
    dists_dtype=tf.float64,  # ensure Cholesky and covariance are computed in float64
    kernel='RBF',
    log_prob_method='precision',  # choose 'precision' for testing the new option.
)

# Unfixed instance: recomputes covariance every time.
gp_unfixed = GPdists(
    dists,
    fixed_params=False,
    full_norm=full_norm,  # will be passed to unfixed precision method
    gp_variance=gp_variance,
    gp_lengthscale=gp_lengthscale,
    gp_mean=gp_mean,
    gp_nugget=gp_nugget,
    psd_control='euclidean',
    dists_dtype=tf.float64,
    kernel='RBF',
    log_prob_method='precision',
)

# ------------------------------
# Create a random parameter vector (matches number of vertices).
# ------------------------------
parameter = tf.random.normal([n_vx], dtype=tf.float32)

# Warm-up (compile tf.function graphs)
_ = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_unfixed._return_log_prob_unfixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_unfixed._return_log_prob_unfixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)

# ------------------------------
# Helper function to time method calls.
# ------------------------------
def time_method(method, instance, param, iterations=5000, **kwargs):
    # Warm-up one call outside timing
    _ = method(param, gp_lengthscale, gp_variance, gp_mean, gp_nugget, **kwargs)
    start = time.time()
    for _ in range(iterations):
        _ = method(param, gp_lengthscale, gp_variance, gp_mean, gp_nugget, **kwargs)
    return (time.time() - start) / iterations

# ------------------------------
# Devices to test (CPU and GPU if available)
# ------------------------------
devices = ['/cpu:0']
if tf.config.list_physical_devices('GPU'):
    devices.append('/gpu:0')

# ------------------------------
# Run tests on each device.
# ------------------------------
for device in devices:
    print("\nTesting on device:", device)
    with tf.device(device):
        # Fixed methods.
        t_fixed_tf = time_method(gp_fixed._return_log_prob_fixed_tf, gp_fixed, parameter)
        t_fixed_prec = time_method(gp_fixed._return_log_prob_fixed_prec, gp_fixed, parameter)
        
        # Unfixed methods.
        t_unfixed_tf = time_method(gp_unfixed._return_log_prob_unfixed_tf, gp_unfixed, parameter)
        t_unfixed_prec = time_method(gp_unfixed._return_log_prob_unfixed_prec, gp_unfixed, parameter)
        
        # Compute log probabilities (for a single call).
        lp_fixed_tf = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        lp_fixed_prec = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        lp_unfixed_tf = gp_unfixed._return_log_prob_unfixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        lp_unfixed_prec = gp_unfixed._return_log_prob_unfixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        
        print("Log probability (Fixed TF):        ", lp_fixed_tf.numpy())
        print("Log probability (Fixed Prec):        ", lp_fixed_prec.numpy())
        print("Log probability (Unfixed TF):        ", lp_unfixed_tf.numpy())
        print("Log probability (Unfixed Prec):      ", lp_unfixed_prec.numpy())
        
        print("\nAverage runtime per call:")
        print("  Fixed TF method:      {:.6f} sec".format(t_fixed_tf))
        print("  Fixed Precision method: {:.6f} sec".format(t_fixed_prec))
        print("  Unfixed TF method:    {:.6f} sec".format(t_unfixed_tf))
        print("  Unfixed Precision method: {:.6f} sec".format(t_unfixed_prec))


Embedding in Euclidean space...
Precomputing covariance matrix...
Precomputing Cholesky decomposition...
Embedding in Euclidean space...

Testing on device: /cpu:0
Log probability (Fixed TF):         -150.07254
Log probability (Fixed Prec):         -150.07259
Log probability (Unfixed TF):         -150.07254
Log probability (Unfixed Prec):       -150.07259

Average runtime per call:
  Fixed TF method:      0.000928 sec
  Fixed Precision method: 0.000895 sec
  Unfixed TF method:    0.000986 sec
  Unfixed Precision method: 0.001044 sec

Testing on device: /gpu:0
Log probability (Fixed TF):         -150.07254
Log probability (Fixed Prec):         -150.07254
Log probability (Unfixed TF):         -150.07254
Log probability (Unfixed Prec):       -150.07254

Average runtime per call:
  Fixed TF method:      0.000993 sec
  Fixed Precision method: 0.001089 sec
  Unfixed TF method:    0.001324 sec
  Unfixed Precision method: 0.001957 sec


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import time
import numpy as np

from braincoder.bprf_mcmc import GPdists, mds_embedding, compute_euclidean_distance_matrix
tfd = tfp.distributions
full_norm = False
# ------------------------------
# Setup: create a dummy distance matrix.
# ------------------------------
n_vx = 50
rand = tf.random.uniform((n_vx, n_vx), minval=0, maxval=1, dtype=tf.float64)
dists = (rand + tf.transpose(rand)) / 2.0
dists = dists - tf.linalg.diag(tf.linalg.diag_part(dists))

# ------------------------------
# Hyperparameters
# ------------------------------
# Note: GPdists expects the distance tensor to be tf.float64 (for Cholesky) so we set that,
# but the parameter vector itself is in tf.float32.
gp_variance = tf.constant(1.0, dtype=tf.float32)
gp_lengthscale = tf.constant(1.0, dtype=tf.float32)
gp_mean = tf.constant(0.0, dtype=tf.float32)
gp_nugget = tf.constant(0.1, dtype=tf.float32)

# ------------------------------
# Instantiate GPdists: fixed and unfixed versions.
# ------------------------------
# Fixed instance: precomputes covariance matrix, Cholesky, and precision.
gp_fixed = GPdists(
    dists,
    fixed_params=True,
    full_norm=True,  # Option for full normalization in precision-based log_prob.
    gp_variance=gp_variance,
    gp_lengthscale=gp_lengthscale,
    gp_mean=gp_mean,
    gp_nugget=gp_nugget,
    psd_control='euclidean',
    dists_dtype=tf.float64,  # ensure Cholesky and covariance are computed in float64
    kernel='RBF',
    log_prob_method='precision',  # choose 'precision' for testing the new option.
)

# Unfixed instance: recomputes covariance every time.
gp_unfixed = GPdists(
    dists,
    fixed_params=False,
    full_norm=True,  # will be passed to unfixed precision method
    gp_variance=gp_variance,
    gp_lengthscale=gp_lengthscale,
    gp_mean=gp_mean,
    gp_nugget=gp_nugget,
    psd_control='euclidean',
    dists_dtype=tf.float64,
    kernel='RBF',
    log_prob_method='precision',
)

# ------------------------------
# Create a random parameter vector (matches number of vertices).
# ------------------------------
parameter = tf.random.normal([n_vx], dtype=tf.float32)

# Warm-up (compile tf.function graphs)
_ = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget, full_norm=True)
_ = gp_unfixed._return_log_prob_unfixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
_ = gp_unfixed._return_log_prob_unfixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget, full_norm=True)

# ------------------------------
# Helper function to time method calls.
# ------------------------------
def time_method(method, instance, param, iterations=100, **kwargs):
    # Warm-up one call outside timing
    _ = method(param, gp_lengthscale, gp_variance, gp_mean, gp_nugget, **kwargs)
    start = time.time()
    for _ in range(iterations):
        _ = method(param, gp_lengthscale, gp_variance, gp_mean, gp_nugget, **kwargs)
    return (time.time() - start) / iterations

# ------------------------------
# Devices to test (CPU and GPU if available)
# ------------------------------
devices = ['/cpu:0']
if tf.config.list_physical_devices('GPU'):
    devices.append('/gpu:0')

# ------------------------------
# Run tests on each device.
# ------------------------------
for device in devices:
    print("\nTesting on device:", device)
    with tf.device(device):
        # Fixed methods.
        t_fixed_tf = time_method(gp_fixed._return_log_prob_fixed_tf, gp_fixed, parameter)
        t_fixed_prec = time_method(gp_fixed._return_log_prob_fixed_prec, gp_fixed, parameter, full_norm=True)
        
        # Unfixed methods.
        t_unfixed_tf = time_method(gp_unfixed._return_log_prob_unfixed_tf, gp_unfixed, parameter)
        t_unfixed_prec = time_method(gp_unfixed._return_log_prob_unfixed_prec, gp_unfixed, parameter, full_norm=True)
        
        # Compute log probabilities (for a single call).
        lp_fixed_tf = gp_fixed._return_log_prob_fixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        lp_fixed_prec = gp_fixed._return_log_prob_fixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget, full_norm=True)
        lp_unfixed_tf = gp_unfixed._return_log_prob_unfixed_tf(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget)
        lp_unfixed_prec = gp_unfixed._return_log_prob_unfixed_prec(parameter, gp_lengthscale, gp_variance, gp_mean, gp_nugget, full_norm=True)
        
        print("Log probability (Fixed TF):        ", lp_fixed_tf.numpy())
        print("Log probability (Fixed Prec):        ", lp_fixed_prec.numpy())
        print("Log probability (Unfixed TF):        ", lp_unfixed_tf.numpy())
        print("Log probability (Unfixed Prec):      ", lp_unfixed_prec.numpy())
        
        print("\nAverage runtime per call:")
        print("  Fixed TF method:      {:.6f} sec".format(t_fixed_tf))
        print("  Fixed Precision method: {:.6f} sec".format(t_fixed_prec))
        print("  Unfixed TF method:    {:.6f} sec".format(t_unfixed_tf))
        print("  Unfixed Precision method: {:.6f} sec".format(t_unfixed_prec))


# bloop

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import numpy as np

tfd = tfp.distributions

# Generate a random positive definite covariance matrix for a 5-dimensional Gaussian.
n = 5
# Create a random matrix and form a covariance matrix by A * A^T.
A = tf.random.normal((n, n), dtype=tf.float64)
cov = tf.matmul(A, A, transpose_b=True) + tf.eye(n, dtype=tf.float64)*1e-3
mean = tf.random.normal((n,), dtype=tf.float64)
x = tf.random.normal((n,), dtype=tf.float64)

# 1. Log probability using TFP's MultivariateNormalFullCovariance:
dist = tfd.MultivariateNormalFullCovariance(loc=mean, covariance_matrix=cov)
log_prob_tfp = dist.log_prob(x)

# 2. Log probability using the full Gaussian formula:
#    log p(x) = -0.5 * [ n*log(2π) + log|cov| + (x-mean)^T cov^-1 (x-mean) ]
def full_gaussian_log_prob(x, mean, cov):
    n = tf.cast(tf.shape(x)[0], cov.dtype)
    diff = x - mean
    inv_cov = tf.linalg.inv(cov)
    log_det_cov = tf.linalg.logdet(cov)
    quad_form = tf.tensordot(diff, tf.linalg.matvec(inv_cov, diff), axes=1)
    # Cast constant 2*pi to the same dtype as cov
    two_pi = tf.constant(2 * math.pi, dtype=cov.dtype)
    return -0.5 * (n * tf.math.log(two_pi) + log_det_cov + quad_form)

log_prob_full = full_gaussian_log_prob(x, mean, cov)

# 3. Log probability using the precision matrix directly:
#    We use the fact that:
#      log p(x) = 0.5 * log|Q| - 0.5 * (x-mean)^T Q (x-mean) - (n/2)*log(2π)
#    where Q = cov^-1.
Q = tf.linalg.inv(cov)
log_det_Q = tf.linalg.logdet(Q)
diff = x - mean
quad_form_precision = tf.tensordot(diff, tf.linalg.matvec(Q, diff), axes=1)
two_pi = tf.constant(2 * math.pi, dtype=cov.dtype)
log_prob_precision = 0.5 * log_det_Q - 0.5 * quad_form_precision - 0.5 * tf.cast(n, cov.dtype) * tf.math.log(two_pi)

print("Log probability (TFP):         ", log_prob_tfp.numpy())
print("Log probability (Full formula):", log_prob_full.numpy())
print("Log probability (Precision):   ", log_prob_precision.numpy())


2025-03-25 12:34:37.433877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-25 12:34:37.433948: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-25 12:34:37.433962: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 12:34:37.447875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-25 12:34:47.452034: I te

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


2025-03-25 12:34:47.943368: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x5596a1f19c90


Log probability (TFP):          -6.446665187826045
Log probability (Full formula): -6.446665187826044
Log probability (Precision):    -6.446665187826045
